In [9]:
import mdptoolbox
import numpy as np
from math import ceil, log
from scipy import optimize

In [84]:
probToState1 = 0.22
valueEstimates = [0.0,-5.2,0.0,25.4,10.6,9.2,12.3]
rews = [-2.4,0.8,4.0,2.5,8.6,-6.4,6.1]

In [85]:
number_of_states = len(valueEstimates)
probs = np.zeros((1, number_of_states, number_of_states))

probs[0, 0, 1] = probToState1
probs[0, 0, 2] = 1 - probToState1
for i in range(2, number_of_states-1):
    probs[0, i, i+1] = 1
probs[0, 1, 3] = 1
probs[0, -1, -1] = 1
probs

array([[[0.  , 0.22, 0.78, 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]]])

In [86]:
# rewards is action and state and also next state based 

rewards = np.zeros((1, number_of_states, number_of_states))

rewards[0,0,1] = rews[0]
rewards[0,0,2] = rews[1]
rewards[0,1,3] = rews[2]
rewards[0,2,3] = rews[3]
for i in range(3, number_of_states-1):
    rewards[0, i, i+1] = rews[i+1]
rewards

array([[[ 0. , -2.4,  0.8,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  4. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  2.5,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  8.6,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. , -6.4,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  6.1],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]]])

In [87]:
vi = mdptoolbox.mdp.ValueIteration(probs, rewards, 1)
vi.run()

In [88]:
vi.V

(11.225999999999999,
 12.299999999999999,
 10.799999999999999,
 8.299999999999999,
 -0.3000000000000007,
 6.1,
 0.0)

In [89]:
### k-step estimate
# if k == 1 then only check the place we can go directly
k_list = []
for i in range(1, number_of_states):
    k_step = mdptoolbox.mdp.ValueIteration(probs, reward=rewards, discount=1, max_iter=i,\
                                   initial_value=valueEstimates)
    k_step.run()
    k_list.append(k_step.V[0])
k_list

[-1.048, 28.326, 22.125999999999998, 14.325999999999999, 23.526, 23.526]

In [90]:
def td_lambda(lam, k_list=k_list, checkScore = vi.V[0]):
    n_state = len(k_list)
    ret = 0
    for i in range(10000):
        if i < n_state:
            comp = (1-lam) * (lam**i) * k_list[i]
        else:
            comp = (1-lam) * (lam**i) * k_list[-1]
        ret += comp
    return abs(ret - checkScore)

In [91]:
best = optimize.newton(td_lambda, x0=0)
best

0.478968723580087